# LLM大数据文本情感分析

最近发现有些人在美团对餐厅评价时，一边打着差评一边给着高分（星星数），因此，美团的真实的星级评分应该比后台数据所计算的平均值低不少。

为了规避此种现象，我们小组开发了此AI大数据分析项目。

本项目运营阿里云平台和阿里巴巴的qwen-max模型进行LLM情感分析。在对评论进行量化之后，综合给予店铺评分。最后用随机森林、支持向量机以及高斯朴素贝叶斯三种算法对训练集店铺数据进行训练（训练集分三个类别：优质店铺、普通店铺和劣质店铺）。

之后通过比较Accuracy、Time Used、Precision Rate、Recall和F1 Score来评估模型。

最终混合模型运用到店铺筛查中，以达到商用的目的。

## 我们运用的商务分析方法

1. 爬取美团评论并收集数据

用户声音分析（VoC）/文本数据分析

通过收集用户公开评价来获取用户需求与感知


2. 对评论进行多维度量化（口味、服务、配送）

多维度用户满意度分析 / 指标体系构建（KPI体系）

将主观评价转为可量化指标，用于度量体验质量


3. AI不确定性建模 + 多算法建模

敏感性分析、预测建模、风险概率分析

使用模型处理不确定性、对结果可信度进行评估


4. 构建模型并验证准确性

模型评估与验证（A/B测试思想、准确率评估）

用数据验证模型是否适用，用于业务落地


5. 用于店铺优良分类

分类与评分体系（评分卡模型）+ 决策支持系统（DSS）方法

将分析结果转化为可执行的商业分类、评级和决策依据


## 导入运行库

In [14]:
import pandas as pd
import cntext as ct
# ct.hello()

## 构造实验数据

In [15]:
df = pd.read_csv('./Process_Data/test.csv')

## 分析函数

In [ ]:

def llm_analysis(text):
    result = ct.llm(text=str(text),
                    prompt= '从口味taste、速度speed、服务service三个维度， 对外卖评论内容进行文本分析， 分别返回不同维度的分值(分值范围-1.0 ~ 1.0)',
                    base_url='https://api.qwen.cloud/v1/chat/completions',
                    api_key='your_api_key_here',
                    model_name='qwen3-max',
                    output_format={'taste': float, 'speed': float, 'service': float}
                               )
    return pd.Series(result)

## 批量运算

In [17]:
df2 = df['comment'].apply(llm_analysis)
res_df = pd.concat([df, df2], axis=1)

## 保存分析结果

In [18]:
res_df.to_csv('result.csv', index=False)
res_df

,comment,taste,speed,service
0,我算是老回头客了，这家的烤鸭量大味道还好，还帮忙撕开！太贴心了。,0.9,0.5,0.8
1,好吃是挺好吃的，就是鸭脖太多了。点；点了半只鸭，里面七八块鸭脖,0.6,0.0,-0.4
2,太难吃了，又咸又干，还一股糊味儿。鸭毛都没清理干净。,-0.9,0.0,-0.6
3,太好吃了捏😋~不就是烤糊了吗！速度太快了捏~不就是送到凉了吗~所以我就分五期给你五星好评吧😡...,0.8,-0.6,-0.7
